#Filtering and sorting in pandas and pyspark

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
import pandas as pd 
df = pd.read_csv('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv', sep='\t')

In [0]:
df.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
sdf = spark.createDataFrame(df)
sdf.show(5)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                null|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                null|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
+--------+--------+--------------------+--------------------+----------+
only showing top 5 rows



#How many products cost more than $10.00?

In [0]:
cleae_it = lambda x:x[1:-1]
df['item_priced'] = df.item_price.apply(cleae_it).astype(float)
df.head()
df[df['item_priced']>10].count()[1]

Out[5]: 1130

In [0]:
df

,order_id,quantity,item_name,choice_description,item_price,item_priced
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39,2.39
1,1,1,Izze,[Clementine],$3.39,3.39
2,1,1,Nantucket Nectar,[Apple],$3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,16.98
...,...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",$11.75,11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",$11.75,11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25,11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75,8.75


In [0]:
sdf = sdf.withColumn('item_priced', substring(sdf['item_price'], 2, 5))
sdf = sdf.withColumn('item_priced',sdf['item_priced'].cast(DoubleType()))
sdf.show()

+--------+--------+--------------------+--------------------+----------+-----------+
|order_id|quantity|           item_name|  choice_description|item_price|item_priced|
+--------+--------+--------------------+--------------------+----------+-----------+
|       1|       1|Chips and Fresh T...|                null|    $2.39 |       2.39|
|       1|       1|                Izze|        [Clementine]|    $3.39 |       3.39|
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |       3.39|
|       1|       1|Chips and Tomatil...|                null|    $2.39 |       2.39|
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |      16.98|
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |      10.98|
|       3|       1|       Side of Chips|                null|    $1.69 |       1.69|
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |      11.75|
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    

In [0]:
sdf[sdf['item_priced']>10].count()

Out[8]: 1130

#Sort by the name of the item


In [0]:
df.sort_values(by='item_name')

,order_id,quantity,item_name,choice_description,item_price,item_priced
3389,1360,2,6 Pack Soft Drink,[Diet Coke],$12.98,12.98
341,148,1,6 Pack Soft Drink,[Diet Coke],$6.49,6.49
1849,749,1,6 Pack Soft Drink,[Coke],$6.49,6.49
1860,754,1,6 Pack Soft Drink,[Diet Coke],$6.49,6.49
2713,1076,1,6 Pack Soft Drink,[Coke],$6.49,6.49
...,...,...,...,...,...,...
2384,948,1,Veggie Soft Tacos,"[Roasted Chili Corn Salsa, [Fajita Vegetables,...",$8.75,8.75
781,322,1,Veggie Soft Tacos,"[Fresh Tomato Salsa, [Black Beans, Cheese, Sou...",$8.75,8.75
2851,1132,1,Veggie Soft Tacos,"[Roasted Chili Corn Salsa (Medium), [Black Bea...",$8.49,8.49
1699,688,1,Veggie Soft Tacos,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice,...",$11.25,11.25


In [0]:
# sdf.orderBy(col('item_name')).show()
sdf.orderBy("item_name").show()

+--------+--------+-----------------+------------------+----------+-----------+
|order_id|quantity|        item_name|choice_description|item_price|item_priced|
+--------+--------+-----------------+------------------+----------+-----------+
|    1664|       1|6 Pack Soft Drink|       [Diet Coke]|    $6.49 |       6.49|
|     129|       1|6 Pack Soft Drink|          [Sprite]|    $6.49 |       6.49|
|    1196|       1|6 Pack Soft Drink|       [Diet Coke]|    $6.49 |       6.49|
|     148|       1|6 Pack Soft Drink|       [Diet Coke]|    $6.49 |       6.49|
|     481|       1|6 Pack Soft Drink|            [Coke]|    $6.49 |       6.49|
|     154|       1|6 Pack Soft Drink|            [Coke]|    $6.49 |       6.49|
|    1248|       1|6 Pack Soft Drink|       [Diet Coke]|    $6.49 |       6.49|
|     168|       1|6 Pack Soft Drink|       [Diet Coke]|    $6.49 |       6.49|
|     774|       1|6 Pack Soft Drink|       [Diet Coke]|    $6.49 |       6.49|
|     182|       1|6 Pack Soft Drink|   

#What was the quantity of the most expensive item ordered?

In [0]:
# df[df['item_priced']==df['item_priced'].max()]
df.sort_values(by='item_priced', ascending=False).head(1)

,order_id,quantity,item_name,choice_description,item_price,item_priced
3598,1443,15,Chips and Fresh Tomato Salsa,NaN,$44.25,44.25


In [0]:
sdf.orderBy(['item_priced'],ascending=False).show(1)#.collect()[0]

+--------+--------+--------------------+------------------+----------+-----------+
|order_id|quantity|           item_name|choice_description|item_price|item_priced|
+--------+--------+--------------------+------------------+----------+-----------+
|    1443|      15|Chips and Fresh T...|              null|   $44.25 |      44.25|
+--------+--------+--------------------+------------------+----------+-----------+
only showing top 1 row



#How many times was a Veggie Salad Bowl ordered?

In [0]:
df[df["item_name"] == "Veggie Salad Bowl"].count()  # Veggie Bowl

Out[13]: order_id              18
quantity              18
item_name             18
choice_description    18
item_price            18
item_priced           18
dtype: int64

In [0]:
# sdf.select('item_name'=="Veggie Salad Bowl")
sdf.filter(col('item_name')=="Veggie Salad Bowl").count()

Out[14]: 18

#How many times did someone order more than one Canned Soda?

In [0]:
ddf = df[df["item_name"]=="Canned Soda"]
ddf[df["quantity"]>1].count()

<command-2548734444638310>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ddf[df["quantity"]>1].count()


Out[15]: order_id              20
quantity              20
item_name             20
choice_description    20
item_price            20
item_priced           20
dtype: int64

In [0]:
sdf.filter((col('item_name')=="Canned Soda") & (col('quantity')>1)).count()

Out[16]: 20